In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import notebook
import datetime

import xgboost as xgb
from sklearn.model_selection import train_test_split
import lightgbm

#한글 깨짐방지
plt.rc('font',family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

C:\Users\User\Anaconda3\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [240]:
path = './data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission  = pd.read_csv(path + 'submission.csv')
merge_df = pd.concat([train,test])

In [3]:
train.shape, test.shape

((3362796, 8), (473392, 8))

In [238]:
test['date'] = pd.to_datetime(test['date'])
gap_check = test.groupby('store_id')['date'].agg(['min','max'])
gap = (gap_check['max']-gap_check['min']).apply(lambda x : x.days)
print(f'test의 최소 길이: {gap.min()},test의 최대길이 : {gap.max()}')

test의 최소 길이: 93,test의 최대길이 : 607


### Data setting
* 거래 취소의 경우 거래금액, 환불금액 2회 체크되므로 거래취소 횟수의 2배만큼 제외해야함
* 날짜기준으로 묶어줄 필요가 있음
* 주말과 요일은 날짜기준으로 결합

-> 시간은 EDA에서 상세히 다시살펴볼것 (새벽시간대 매출이 높은 곳이라던지)

In [241]:
# train 중 전체 거래일이 100일 이하인경우 학습에 사용불가
train['date'] = pd.to_datetime(train['date'])
gap_check = train.groupby('store_id')['date'].agg(['min','max'])
gap = (gap_check['max']-gap_check['min']).apply(lambda x : x.days)
print(f'train의 최소 길이: {gap.min()},train의 최대길이 : {gap.max()}')
del_id = gap[gap<100].index
clean_train = train[~(train.store_id.isin(del_id))]

train의 최소 길이: 0,train의 최대길이 : 729


In [134]:
print(f'기본 트레인 길이 : {train.shape[0]}, 기본트레인 상점수 : {train.store_id.nunique()}')
print(f'clean 트레인 길이 : {clean_train.shape[0]}, clean 트레인 상점수 : {clean_train.store_id.nunique()}')

기본 트레인 길이 : 3362796, 기본트레인 상점수 : 1775
clean 트레인 길이 : 3345678, clean 트레인 상점수 : 1641


In [4]:
#전체날짜의 휴일 요일
all_dates = merge_df.drop_duplicates('date').sort_values('date')
all_dates = all_dates[['date','days_of_week','holyday']]
all_dates.reset_index(drop=True, inplace=True)
all_dates['date'] = pd.to_datetime(all_dates['date'])

In [215]:
clean_train.loc[clean_train.amount <0,'refund'] = 1
sample_df = clean_train.groupby(['date','store_id']).agg({'amount':['sum','count'],'refund':'sum'}).reset_index()
sample_df.columns = ['date','store_id','amount_sum','amount_count','refund_sum']
sample_df['date'] = pd.to_datetime(sample_df['date'])
sample_df['temp_date'] = sample_df['date']

#거래횟수당 평균 거래금액 
sample_df['trans_amount'] = sample_df['amount_sum']/(sample_df['amount_count']-sample_df['refund_sum']*2)
sample_df['trans_amount'].fillna(0,inplace= True)

#날짜병합
sample_df = pd.merge(sample_df,all_dates)

sample_df.set_index("date",inplace=True)

C:\Users\User\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [271]:
store_id_list = sample_df.store_id.unique()
fin_ls =[]
for store_id in notebook.tqdm(store_id_list[:3]):
    feat = dict()
    
    store = sample_df[sample_df.store_id == store_id]
    
    weekly = store[['days_of_week','amount_sum']].groupby('days_of_week').agg(['sum','count'])
    weekly.columns = ['weekly_amount_sum','weekly_amount_count']
    weekly_dict = weekly.to_dict()

    holyday = store[['holyday','amount_sum']].groupby('holyday').agg(['sum','count'])
    holyday.columns = ['holyday_amount_sum','holyday_amount_count']
    holyday_dict = holyday.to_dict()
    
    feat.update(weekly_dict)
    feat.update(holyday_dict)
    
    store = store.asfreq('D',fill_value = 0) #일자별로 열추가 
    store['temp_date'] = store.index
    store['store_id'] = store_id

    # 7일, 15일, 30일간의 이동평균 계산 
    store['MA7'] = store['amount_sum'].rolling('7D').mean()
    store['MA15'] = store['amount_sum'].rolling('15D').mean()
    store['MA30'] = store['amount_sum'].rolling('30D').mean()
    
    # train set인 경우 마지막 100일은 y값으로 저장되어야함
    # store_x는 100일전까지 기간 데이터 
    if isTrain:
        store_y = store.last("100D")
        y = store_y.amount_sum.sum()
        store_x = store[store.temp_date < store_y.iloc[0].temp_date]
    #test는 전체
    else : 
        y = 0
        store_x = store[:]
        
#     feat = dict()
    feat['store_id'] = store_id
    feat['y'] = y
    feat['daily_mean'] = store_x.amount_sum.mean()
    feat['daily_mid'] = store_x.amount_sum.median()
    feat['amount_count'] = store.amount_count.sum()
    feat['refund_count'] = store.refund_sum.sum()
    feat['daily_trans_amount'] = store.trans_amount.sum() / store[store.trans_amount>0].shape[0]
    
    feat['last7D_mean'] = store_x.last('7D').amount_sum.mean()
    feat['last15D_mean'] = store_x.last('15D').amount_sum.mean()
    feat['last30D_mean'] = store_x.last('30D').amount_sum.mean()
    
    feat['last7D_median'] = store_x.last('7D').amount_sum.median()
    feat['last15D_median'] = store_x.last('15D').amount_sum.median()
    feat['last30D_median'] = store_x.last('30D').amount_sum.median()
    
    feat['last7D_sum'] = store_x.last('7D').amount_sum.sum()
    feat['last15D_sum'] = store_x.last('15D').amount_sum.sum()
    feat['last30D_sum'] = store_x.last('30D').amount_sum.sum()
    
    feat['last7D_MA7_mean'] = store_x.last('7D').MA7.mean()
    feat['last15D_MA15_mean'] = store_x.last('15D').MA15.mean()
    feat['last30D_MA30_mean'] = store_x.last('30D').MA30.mean()
    
    feat['last7D_MA7_median'] = store_x.last('7D').MA7.median()
    feat['last15D_MA15_median'] = store_x.last('15D').MA15.median()
    feat['last30D_MA30_median'] = store_x.last('30D').MA30.median()
    
    feat['last7D_MA7_sum'] = store_x.last('7D').MA7.sum()
    feat['last15D_MA15_sum'] = store_x.last('15D').MA15.sum()
    feat['last30D_MA30_sum'] = store_x.last('30D').MA30.sum()
    
    fin_ls.append(feat)

### 주별, 주말별 계산 어쩌냐

In [251]:
weekly = store[['days_of_week','amount_sum']].groupby('days_of_week').agg(['sum','count'])
weekly.columns = ['weekly_amount_sum','weekly_amount_count']
weekly_dict = weekly.to_dict()

holyday = store[['holyday','amount_sum']].groupby('holyday').agg(['sum','count'])
holyday.columns = ['holyday_amount_sum','holyday_amount_count']
holyday_dict = holyday.to_dict()

In [278]:
feat.update(weekly['weekly_amount_sum'].to_dict())

In [228]:
store_x[store_x.amount_sum>0].shape[0]

1

In [230]:
store_x

,store_id,amount_sum,amount_count,refund_sum,temp_date,trans_amount,days_of_week,holyday,MA7,MA15,MA30
date,,,,,,,,,,,
2018-04-21,1163,545,1,0.0,2018-04-21,545.0,5,0,545.0,545.0,545.0
2018-04-22,1163,0,0,0.0,2018-04-22,0.0,0,0,272.5,272.5,272.5


In [188]:
store

,store_id,amount_sum,amount_count,refund_sum,temp_date,days_of_week,holyday,trans_amount,MA7,MA15,MA30


In [189]:
store = sample_df[sample_df.store_id == 8]

In [164]:
store = sample_df[sample_df.store_id == store_id]

In [174]:
store.trans_amount.sum()

65714.6097208347

In [173]:
store[store.trans_amount>0].shape[0]

531

In [151]:
store = store.asfreq('D',fill_value = 0)

In [152]:
store['temp_date'] = store.index
store['store_id'] = store_id

In [153]:
store

,store_id,amount_sum,amount_count,refund_sum,temp_date
date,,,,,
2016-08-10,32,140,1,0.0,2016-08-10
2016-08-11,32,0,0,0.0,2016-08-11
2016-08-12,32,0,0,0.0,2016-08-12
2016-08-13,32,60,1,0.0,2016-08-13
2016-08-14,32,0,0,0.0,2016-08-14
...,...,...,...,...,...
2018-07-26,32,884,4,0.0,2018-07-26
2018-07-27,32,549,5,0.0,2018-07-27
2018-07-28,32,709,4,0.0,2018-07-28


In [83]:
store_y = store.last("100D")

In [85]:
store_y.amount_sum.sum()

510